In [ ]:
# ARMA Time Series Prediction in Python
# ====================================

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# 1. Generate a time series with ARMA process
# ------------------------------------------
# We'll create a time series using ARMA(2,1) process
# AR parameters: phi = [0.75, -0.25]
# MA parameters: theta = [0.65]

# Define the AR and MA parameters
ar_params = np.array([0.75, -0.25])
ma_params = np.array([0.65])

# Create ARMA process coefficients
# Note: The first coefficient is 1 for both AR and MA
ar = np.r_[1, -ar_params]  # Add the leading 1
ma = np.r_[1, ma_params]   # Add the leading 1

# Generate the sample with 500 observations
n_samples = 500
y = arma_generate_sample(ar=ar, ma=ma, nsample=n_samples)

# Create a date range for the time series
dates = pd.date_range(start='2020-01-01', periods=n_samples, freq='D')
ts = pd.Series(y, index=dates)

# 2. Exploratory Data Analysis
# ---------------------------
plt.figure(figsize=(14, 6))
plt.plot(ts)
plt.title('Generated ARMA(2,1) Time Series')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True)
plt.show()

# Plot ACF and PACF to identify model order
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# ACF plot
plot_acf(ts, lags=40, ax=ax1)
ax1.set_title('Autocorrelation Function (ACF)')

# PACF plot
plot_pacf(ts, lags=40, ax=ax2)
ax2.set_title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

# 3. Train-Test Split
# ------------------
# Split the time series into training and testing sets
train_size = int(len(ts) * 0.8)
train, test = ts[:train_size], ts[train_size:]

print(f"Training set size: {len(train)} observations")
print(f"Testing set size: {len(test)} observations")

# Plot the train-test split
plt.figure(figsize=(14, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Testing Data')
plt.title('Train-Test Split')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# 4. Model Fitting
# ---------------
# We know it's an ARMA(2,1) process, but let's pretend we don't
# Let's fit an ARIMA(2,0,1) which is equivalent to ARMA(2,1)

# Fit the model
model = ARIMA(train, order=(2, 0, 1))
model_fit = model.fit()

# Display model summary
print("Model Summary:")
print(model_fit.summary())

# 5. Model Diagnostics
# ------------------
# Plot residuals
residuals = model_fit.resid
plt.figure(figsize=(14, 7))

# Residuals over time
plt.subplot(211)
plt.plot(residuals)
plt.title('Residuals')
plt.xlabel('Date')
plt.ylabel('Residual Value')
plt.grid(True)

# Histogram of residuals with density plot
plt.subplot(212)
sns.histplot(residuals, kde=True)
plt.title('Histogram of Residuals')
plt.xlabel('Residual Value')
plt.grid(True)

plt.tight_layout()
plt.show()

# ACF of residuals
plt.figure(figsize=(14, 4))
plot_acf(residuals, lags=40)
plt.title('ACF of Residuals')
plt.grid(True)
plt.show()

# Ljung-Box test for autocorrelation in residuals
lb_test = acorr_ljungbox(residuals, lags=[10, 20, 30], return_df=True)
print("Ljung-Box Test for Residual Autocorrelation:")
print(lb_test)

# 6. In-sample Predictions
# ----------------------
# Get in-sample predictions
in_sample_pred = model_fit.predict(start=0, end=len(train)-1)

# Plot actual vs predicted values
plt.figure(figsize=(14, 6))
plt.plot(train, label='Actual')
plt.plot(in_sample_pred, label='Predicted', alpha=0.7)
plt.title('In-sample Predictions vs Actual Values')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# Calculate in-sample performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
rmse_in = np.sqrt(mean_squared_error(train, in_sample_pred))
mae_in = mean_absolute_error(train, in_sample_pred)

print(f"In-sample RMSE: {rmse_in:.4f}")
print(f"In-sample MAE: {mae_in:.4f}")

# 7. Out-of-sample Forecasting
# --------------------------
# Forecast for the test period
forecast_steps = len(test)
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int(alpha=0.05)  # 95% confidence interval

# Plot forecasts against actual test data
plt.figure(figsize=(14, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Actual Test Data')
plt.plot(forecast_mean, label='Forecast', color='red')
plt.fill_between(forecast_mean.index, 
                 forecast_ci.iloc[:, 0], 
                 forecast_ci.iloc[:, 1], 
                 color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title('ARMA(2,1) Forecast vs Actual Values')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# Calculate out-of-sample performance metrics
rmse_out = np.sqrt(mean_squared_error(test, forecast_mean))
mae_out = mean_absolute_error(test, forecast_mean)

print(f"Out-of-sample RMSE: {rmse_out:.4f}")
print(f"Out-of-sample MAE: {mae_out:.4f}")

# 8. Re-fit on Full Dataset and Future Forecasting
# ----------------------------------------------
# Fit the model on the full dataset
full_model = ARIMA(ts, order=(2, 0, 1))
full_model_fit = full_model.fit()

# Forecast future values
future_steps = 100
future_forecast = full_model_fit.get_forecast(steps=future_steps)
future_mean = future_forecast.predicted_mean
future_ci = future_forecast.conf_int(alpha=0.05)

# Plot historical data and future forecast
plt.figure(figsize=(14, 6))
plt.plot(ts, label='Historical Data')
plt.plot(future_mean, label='Future Forecast', color='red')
plt.fill_between(future_mean.index, 
                 future_ci.iloc[:, 0], 
                 future_ci.iloc[:, 1], 
                 color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title('ARMA(2,1) Future Forecast')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# 9. Analyze Model Parameters
# -------------------------
print("\nEstimated Model Parameters:")
print(f"AR parameters: {model_fit.arparams}")
print(f"MA parameters: {model_fit.maparams}")
print(f"True AR parameters: {ar_params}")
print(f"True MA parameters: {ma_params}")

# 10. Model Comparison (Optional)
# ----------------------------
# Let's fit a few other ARMA models and compare AIC/BIC

models = []
orders = [(1,0,0), (0,0,1), (1,0,1), (2,0,0), (0,0,2), (2,0,1), (1,0,2), (2,0,2)]

for order in orders:
    try:
        temp_model = ARIMA(train, order=order)
        temp_model_fit = temp_model.fit()
        models.append({
            'order': order,
            'aic': temp_model_fit.aic,
            'bic': temp_model_fit.bic
        })
        print(f"ARIMA{order} - AIC: {temp_model_fit.aic:.2f}, BIC: {temp_model_fit.bic:.2f}")
    except:
        print(f"Error fitting ARIMA{order}")

# Convert to DataFrame for easier comparison
models_df = pd.DataFrame(models)
print("\nModel Comparison:")
print(models_df.sort_values('aic'))

# Plot AIC and BIC for different models
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.bar([str(x['order']) for x in models], [x['aic'] for x in models])
plt.title('AIC for Different ARMA Models')
plt.xlabel('Model Order (p,d,q)')
plt.ylabel('AIC')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar([str(x['order']) for x in models], [x['bic'] for x in models])
plt.title('BIC for Different ARMA Models')
plt.xlabel('Model Order (p,d,q)')
plt.ylabel('BIC')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()